In [6]:
with open('../../key.txt', 'r') as f:
    key_file = f.read()

key_file = key_file.split('\n')
keys = {}
for key in key_file:
    key = key.split(':')
    keys[key[0]] =  key[1]

In [7]:
# Load OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = keys['OPENAI_API_KEY']

# Test between the Summarization Task by Langchain & Guidance
- Detail:
    - Objects: test the performance of summarization between Langchain & Guidance
    - BenchMark: Runtime
    - LLM: OpenAI API - "text-davinci-003"
    - Method: Map-Reduce
        1. Split text into chunks
        2. Do summarization on each chunks and get summaries
        3. Do final summarization on the summaries

- Langchain:
    - using map_reduce_chain

- Guidance:
    - using {{#each}}
    - using {{#geneach}}

## Data Preparation
- mock web text
- text splitting & processing

### Mock Web Text

In [1]:
text1_content = """
GPT-3 vs. BERT: Comparing the Two Most Popular Language Models
Natural language processing (NLP) has come a long way over the past few years. With the development of powerful new models such as GPT-3 and BERT, it's now possible to create sophisticated applications that can understand and interact with human language.
However, what went viral as a disruptive chatbot with ChatGPT, suddenly became a contest of language models to power AI content. So, we decided to oppose GPT-3 vs. BERT to understand their differences and similarities, explore their capabilities, and discuss some of the tools that use them.
GPT-3 (Generative Pre-trained Transformer 3) is an autoregressive language model developed by OpenAI. It was trained on a dataset of 45TB of text data from sources such as Wikipedia, books, and webpages. The model is capable of generating human-like text when given a prompt. It can also be used for tasks such as question answering, summarization, translation, and more.
Several AI content writing tools currently use GPT-3, such as:
BERT (Bidirectional Encoder Representations from Transformers) is another popular language model developed by Google AI. Unlike GPT-3, BERT is a bidirectional transformer model, which considers both left and right context when making predictions. This makes it better suited for sentiment analysis or natural language understanding (NLU) tasks.
BERT serves as the base for a number of services, like:
The most obvious difference between GPT-3 and BERT is their architecture. As mentioned above, GPT-3 is an autoregressive model, while BERT is bidirectional. While GPT-3 only considers the left context when making predictions, BERT takes into account both left and right context. This makes BERT better suited for tasks such as sentiment analysis or NLU, where understanding the full context of a sentence or phrase is essential.
Another difference between the two models lies in their training datasets. While both models were trained on large datasets of text data from sources like Wikipedia and books, GPT-3 was trained on 45TB of data, while BERT was trained on 3TB of data. So, GPT-3 has access to more information than BERT, which could give it an edge in specific tasks such as summarization or translation, where access to more data can be beneficial.
Finally, there are differences in terms of size as well. While both models are very large (GPT-3 has 1.5 billion parameters while BERT has 340 million parameters), GPT-3 is significantly larger than its predecessor due to its much more extensive training dataset size (470 times bigger than the one used to train BERT).
Despite their differences in architecture and training datasets size, there are also some similarities between GPT-3 and BERT:
They use the Transformer architecture to learn context from textual-based datasets using attention mechanisms.
They are unsupervised learning models (they don’t require labeled data for training).
They can perform various NLP tasks such as question answering, summarization, or translation with varying degrees of accuracy, depending on the task.
Both GPT-3 and BERT have been shown to perform well on various NLP tasks, including question answering, summarization, or translation, with varying degrees of accuracy depending on the task at hand.
However, due to its larger training dataset size, GPT-3 tends to outperform its predecessor in certain tasks, such as summarization or translation, where having access to more data can be beneficial.
On other tasks, such as sentiment analysis or NLU, BERT tends to do better due to its bidirectional nature, which allows it to take into account both left and right context when making predictions. In contrast, GPT -3 only considers left context when predicting words or phrases in a sentence.
The bottom line is that GPT-3 and BERT have proven themselves valuable tools for performing various NLP tasks with varying degrees of accuracy. However, due to their differences in architecture and training dataset size, each model is better suited for certain tasks than others.
For example, GPT-3 is better suited for summarization or translation, while BERT is more beneficial for sentiment analysis or NLU. Ultimately, the choice between the two models will depend on your specific needs and which task you are looking to accomplish.
"""

text2_content = """
Stack Exchange network consists of 181 Q&A communities including Stack Overflow, the largest, most trusted online community for developers to learn, share their knowledge, and build their careers.
                                        Learn more about Stack Overflow the company, and our products.
Data Science Stack Exchange is a question and answer site for Data science professionals, Machine Learning specialists, and those interested in learning more about the field. It only takes a minute to sign up.
In the BERT paper, I learnt that BERT is encoder-only model, that is it involves only transformer encoder blocks.
In the GPT paper, I learnt that GPT is decoder-only model, that is it involves only transformer decoder blocks.
I was guessing whats the difference. I know following difference between encoder and decoder blocks: GPT Decoder looks only at previously generated tokens and learns from them and not in right side tokens. BERT Encoder gives attention to tokens on both sides.
Q1. GPT2,3 focuses on new/one/zero short learning. Cant we build new/one/zero short learning model with encoder-only architecture like BERT?
Q2. Huggingface Gpt2Model contains forward() method. I guess, feeding single data instance to this method is like doing one shot learning?
Q4. Apart from being decoder-only and encoder-only, auto-regressive vs non-auto-regressive and whether or not accepting tokens generated so far as input, what high level architectural / conceptual differences GPT and BERT have?
To start with your last question: you correctly say that BERT is an encoder-only model trained with the masked language-modeling objective and operates non-autoregressively. GPT-2 is a decode-only model trained using the left-to-right language objective and operates autoregressively. Other than that, there are only technical differences in hyper-parameters, but no other conceptual differences.
BERT (other masked LMs) could also be used for zero- or few-shot learning, but in a slightly different way. There is a method called PET (Pattern-
Exploiting Training). It uses the language modeling abilities of BERT via templates. E.g., for sentiment analysis, you can do something like:
Working with the GPT-2 model is not that straightforward as with BERT. Calling the forward method returns the hidden states of GPT-2 given the input you provided that can be further used in a model. You can use hidden states of GPT-2 as contextual embeddings, the same way that you the output of BERT, however, this is not how GPT-2 is usually used.
The usual way of using GPT-2 sampling from the model. This means that you provide a prompt (as plain text) and hope that the model will continue in a reasonable way. There are many tutorials on how to generate from the GPT-2 models, e.g., this blog post by Huggingface.
By clicking “Post Your Answer”, you agree to our terms of service and acknowledge that you have read and understand our privacy policy and code of conduct.
Not the answer you're looking for? Browse other questions tagged machine-learningnlpberttransformergpt or ask your own question.
YA novel about a girl in dystopian society who gets surgically altered to be a double for a dead girl
Is lying in an application for a job at a private company, signed under penalty of perjury, prosecutable as perjury?
DeSantis' "US Constitution’s 'leverage points'... to exercise the 'true scope' of presidential power". Something new or based on existing theories?
Site design / logo © 2023 Stack Exchange Inc; user contributions licensed under CC BY-SA.                    rev 2023.5.26.43462
By clicking “Accept all cookies”, you agree Stack Exchange can store cookies on your device and disclose information in accordance with our Cookie Policy."""

### Text Splitting & Processing
- Using RecursiveCharacterTextSplitter in Langchain 
- Add Source at the end of each chunk 

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 20,
)

texts1 = text_splitter.split_text(text1_content)
texts2 = text_splitter.split_text(text2_content)

for i in range(len(texts1)):
    texts1[i] = texts1[i] + '\n' + 'Come from: #url1 \n'
    
for i in range(len(texts2)):
    texts2[i] = texts2[i] + '\n' + 'Come from: #url2 \n'

texts = texts1 + texts2

## Summarization with Langchain
- Test on RTX3090 using 43.535649061203s

In [16]:
from langchain import OpenAI, PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

docs = [Document(page_content=t) for t in texts]

openai_llm = OpenAI(
    temperature = 0
)

### Templating

In [17]:
summarize_template = """Write a summary of the following text:
You must include what the text if from in the end of the summary, for example, [from url1] or [from url2]
{text}

Summary: 
"""

summarize_prompt = PromptTemplate(
    template = summarize_template,
    input_variables = ["text"]
)

combine_template = """Write a summary in detail of the following text:
Each bullet point should start with a title in the format of [title], and end with its source in the format of [url1, url2 ...]
Not all of the text are relevant, you should not summarize the irrelevant text.

Example:
[Make Decision] When you get the error message, you should make a decision. [url3, url6]

text:
<{text}>

Summary (in detail):
"""

combine_prompt = PromptTemplate(
    template = combine_template,
    input_variables = ["text"]
)

### Chain

In [18]:
map_reduce_chain = load_summarize_chain(
    llm = openai_llm,
    chain_type = "map_reduce",
    map_prompt = summarize_prompt,
    combine_prompt = combine_prompt,
    combine_document_variable_name = "text",
    verbose = True
)

### Test

In [19]:
import time

start = time.time()
summary = map_reduce_chain.run(docs)
end = time.time()

print(end-start)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a summary of the following text:
You must include what the text if from in the end of the summary, for example, [from url1] or [from url2]
GPT-3 vs. BERT: Comparing the Two Most Popular Language Models
Natural language processing (NLP) has come a long way over the past few years. With the development of powerful new models such as GPT-3 and BERT, it's now possible to create sophisticated applications that can understand and interact with human language.
However, what went viral as a disruptive chatbot with ChatGPT, suddenly became a contest of language models to power AI content. So, we decided to oppose GPT-3 vs. BERT to understand their differences and similarities, explore their capabilities, and discuss some of the tools that use them.
GPT-3 (Generative Pre-trained Transformer 3) is an autoregressive language model developed by OpenAI. It was trained on a dataset of 4

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a summary in detail of the following text:
Each bullet point should start with a title in the format of [title], and end with its source in the format of [url1, url2 ...]
Not all of the text are relevant, you should not summarize the irrelevant text.

Example:
[Make Decision] When you get the error message, you should make a decision. [url3, url6]

text:
<This text compares two of the most popular language models in natural language processing (NLP): GPT-3 and BERT. GPT-3 is an autoregressive language model developed by OpenAI, trained on 45TB of text data from sources such as Wikipedia, books, and webpages. It is capable of generating human-like text when given a prompt and can be used for tasks such as question answering, summarization, translation, and more. BERT is also a powerful language model, used for tasks such as sentiment analysis and question an

## Summarization with Guidance

### Using {{#each}} command
- Test on RTX3090 using 97.12837052345276s

In [12]:
import guidance

guidance.llms.OpenAI.cache.clear()
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

In [13]:
summary_list = []
def add_below(text):
    summary_list.append(text)

summarization_template = """
Write a concise summary on the content. 
You must include the source of text in the end of the summary, 
for example, [from url1] or [from url2]

{{~#each text_list}}
{{#block hidden=True}}
<Content: {{this}}>
Summary: {{add_below (gen 'summary' temperature=0)}}
{{/block}}
{{/each}}
"""

summary_program = guidance(
    template = summarization_template
)

In [14]:
import time

start = time.time()
executed_program = summary_program(text_list = texts, add_below = add_below)
end = time.time()

print(end-start)

Stop program 
Write a concise summary on the content. 
You must include the source of text in the end of the summary, 
for example, [from url1] or [from url2]

97.12837052345276


In [15]:
summary_list

[' GPT-3 and BERT are two of the most popular language models used in natural language processing (NLP). GPT-3 is an autoregressive language model developed by OpenAI, trained on 45TB of text data from sources such as Wikipedia, books, and webpages. It can generate human-like text when given a prompt and can be used for tasks such as question answering, summarization, translation, and more. BERT is a transformer-based language model developed by Google, trained on a large corpus of text data. It can be used for tasks such as sentiment analysis, question answering, and text classification. This article compares the two models and discusses the tools that use them. [from url1]',
 ' GPT-3 and BERT are two popular AI content writing tools. GPT-3 is an autoregressive model while BERT is bidirectional, taking into account both left and right context. This makes BERT better suited for tasks such as sentiment analysis or natural language understanding. BERT serves as the base for a number of s

### Using {{#geneach}} command
- Test on RTX3090 using 79.16032338142395s

In [8]:
import guidance

guidance.llms.OpenAI.cache.clear()
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [9]:
summary_list = []
def add_below(text):
    summary_list.append(text)

summarization_template = """
{{#system~}}
Write a concise summary on the following text. You must include the source of text in the end of the summary, for example, [from url1] or [from url2]
{{~/system}}

{{~#geneach 'summaries' stop=False}}
{{#user~}}
{{set 'this.usr_text' (await 'usr_text')}}
{{~/user}}

{{#assistant~}}
{{add_below (gen 'this.summary' temperature=0 hidden=True)}}
{{~/assistant}}
{{~/geneach}}
"""



summary_program = guidance(
    template = summarization_template,
)

In [10]:
import time

start = time.time()
for text in texts:
    summary_program(usr_text = text, add_below = add_below)
end = time.time()

print(end-start)

Stop program system Write a concise summary on the following text. You must include the source of text in the end of the summary, for example, [from url1] or [from url2] user By clicking “Post Your Answer”, you agree to our terms of service and acknowledge that you have read and understand our privacy policy and code of conduct.
Not the answer you're looking for? Browse other questions tagged machine-learningnlpberttransformergpt or ask your own question.
YA novel about a girl in dystopian society who gets surgically altered to be a double for a dead girl
Is lying in an application for a job at a private company, signed under penalty of perjury, prosecutable as perjury?
DeSantis' "US Constitution’s 'leverage points'... to exercise the 'true scope' of presidential power". Something new or based on existing theories?
Site design / logo © 2023 Stack Exchange Inc; user contributions licensed under CC BY-SA. rev 2023.5.26.43462
By clicking “Accept all cookies”, you agree Stack Exchange can store cookies on your device and disclose information in accordance with our Cookie Policy.
Come from: #url2 
 assistant user {{set 'summaries[-1].usr_text' (await 'usr_text')}} {{#assistant~}} 
 {{add_below (gen 'summaries[-1].summary' temperature=0 hidden=True)}} 
 {{~/assistant}} 
 {{~#geneach 'summaries' stop=False}} 
 {{#user~}} 
 {{set 'this.usr_text' (await 'usr_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{add_below (gen 'this.summary' temperature=0 hidden=True)}} 
 {{~/assistant}} 
 {{~/geneach}}

79.16032338142395


In [11]:
summary_list

['The article discusses the differences and similarities between two popular language models, GPT-3 and BERT, which have revolutionized natural language processing (NLP). GPT-3 is an autoregressive language model developed by OpenAI, trained on a massive dataset of text data from various sources. It can generate human-like text and perform tasks such as question answering, summarization, and translation. The article compares the capabilities of GPT-3 and BERT and discusses some of the tools that use them. [from #url1]',
 'The article discusses the differences between GPT-3 and BERT, two popular language models used in AI content writing tools. While GPT-3 is an autoregressive model that only considers the left context when making predictions, BERT is a bidirectional transformer model that takes into account both left and right context, making it better suited for tasks such as sentiment analysis or natural language understanding. [from #url1]',
 "GPT-3 and BERT are two natural language